In [1]:
from jupyter_imports import *

In [2]:
%%bash 
source ~/.bashrc
#cache_config

In [3]:
dirname = os.path.join(os.sep, 'home','david','config','prod')
original_risk = pd.read_csv(os.path.join(os.sep, dirname ,'all_risk.csv'),parse_dates=['time'])
original_pnl = pd.read_csv(os.path.join(os.sep, dirname,'all_pnl.csv'),parse_dates=['time','start_time'])

In [4]:
exchange = 'ftx'
subaccount = 'debug'
end = datetime.utcnow().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 7)
pnl = original_pnl[(original_pnl['time']>start)&(original_pnl['time']<end)&(original_pnl['subaccount']==subaccount)&(original_pnl['exchange']==exchange)]
risk = original_risk[(original_risk['time']>start)&(original_risk['time']<end)&(original_pnl['subaccount']==subaccount)&(original_pnl['exchange']==exchange)]

/tmp/ipykernel_2353112/1667510151.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [5]:
def category(event_type):
    if event_type in ['future_trade','spot_trade','txFee']: return 'trades' 
    if event_type in ['funding','borrow']: return 'carry' 
    if event_type in ['delta','IR01']: return 'risk' 
    if event_type in ['unexplained']: return 'unexplained'
pnl['category'] = pnl['event_type'].apply(category)

# Cumulative pnl  by category

In [6]:
pnl.pivot_table(index='time',columns='category',values='USDamt').cumsum().iplot()

In [7]:
original_pnl.columns

Index(['Unnamed: 0', 'time', 'coin', 'USDamt', 'event_type', 'attribution',
       'end_mark', 'start_time', 'underlying', 'exchange', 'subaccount'],
      dtype='object')

# risk by coin

In [8]:
risk[(risk['event_type']=='delta')&(risk['attribution'].apply(lambda f: f!='USD' and 'PERP' not in f))].pivot_table(index='time',columns='attribution',values='usdAmt',aggfunc=sum).iplot()

# PV

In [9]:
 risk[risk['event_type']=='PV'].pivot_table(index='time',columns='attribution',values='usdAmt',aggfunc=sum).iplot()